# 대화 인터페이스 - Titan LLM을 사용한 챗봇

> 이 노트북은 **SageMaker Studio의** `Data Science 3.0` 커널과 잘 작동합니다

> **SageMaker Notebook Instance** 를 이용해 실습을 진행하신다면, **JupyterLab이 아닌 Jupyter**에서 실행하시기 바랍니다.

이 노트북에서는 Amazon Bedrock의 Foundational Model을 사용하여 챗봇을 구축합니다. 이 사용 사례에서는 챗봇 구축을 위한 FM으로 Titan을 사용합니다.

## 개요

챗봇 및 가상 비서와 같은 대화 인터페이스를 사용하여 고객의 사용자 경험을 향상할 수 있습니다. 챗봇은 자연어 처리(NLP) 및 기계 학습 알고리즘을 사용하여 사용자의 질문을 이해하고 응답합니다. 챗봇은 고객 서비스, 영업, 전자상거래 등 다양한 애플리케이션에서 사용자에게 빠르고 효율적인 응답을 제공하기 위해 사용될 수 있습니다. 웹사이트, 소셜 미디어 플랫폼, 메시징 앱 등 다양한 채널을 통해 액세스할 수 있습니다.


## Amazon Bedrock을 사용하는 챗봇

![Amazon Bedrock - Conversational Interface](./images/chatbot_bedrock.png)

## 사용 사례

1. **챗봇(기본)** - FM 모델을 사용한 Zero Shot 챗봇
2. **프롬프트를 사용하는 챗봇** - 템플릿(Langchain) - 프롬프트 템플릿에 일부 컨텍스트가 제공되는 챗봇
3. **페르소나가 있는 챗봇** - 정의된 역할이 있는 챗봇입니다. 예, 커리어 코치와 인간 상호 작용
4. **컨텍스트 인식(contextual-aware) 챗봇** - 임베딩을 생성하여 외부 파일을 통해 컨텍스트를 전달합니다.

## Amazon Bedrock을 사용하여 Chatbot을 구축하기 위한 Langchain 프레임워크
챗봇과 같은 대화형 인터페이스에서는 단기적으로나 장기적으로 이전 상호 작용을 기억하는 것이 매우 중요합니다.

LangChain은 두 가지 형태로 메모리 구성 요소를 제공합니다. 첫째, LangChain은 이전 채팅 메시지를 관리하고 조작하기 위한 도우미(helper) 유틸리티를 제공합니다. 이는 모듈식으로 설계되었으며 사용 방법에 관계없이 유용합니다. 둘째, LangChain은 이러한 유틸리티를 체인에 통합하는 쉬운 방법을 제공합니다.
이를 통해 다양한 유형의 추상화를 쉽게 정의하고 상호 작용할 수 있으므로 강력한 챗봇을 쉽게 구축할 수 있습니다.

## 맥락(컨텍스트)를 고려한 챗봇 구축 - 핵심 요소

컨텍스트 인식(contextual-aware) 챗봇을 구축하는 첫 번째 프로세스는 컨텍스트에 대한 **임베딩을 생성**하는 것입니다. 일반적으로 임베딩 모델을 통해 실행되고 일종의 벡터 저장소에 저장될 임베딩을 생성하는 수집 프로세스가 있습니다. 이 예에서는 이를 위해 GPT-J 임베딩 모델을 사용하고 있습니다.

![Embeddings](./images/embeddings_lang.png)

두 번째 프로세스는 사용자 요청 오케스트레이션, 상호 작용, 호출 및 결과 반환입니다.

![Chatbot](./images/chatbot_lang.png)

## Architecture [Context Aware Chatbot]
![4](./images/context-aware-chatbot.png)

## 설정

이 노트북의 나머지 부분을 실행하기 전에 아래 셀을 실행하여 (필요한 라이브러리가 설치되어 있는지 확인하고) Bedrock에 연결해야 합니다.

설정 작동 방식 및 ⚠️ **변경이 필요한지 여부**에 대한 자세한 내용은 [Bedrock boto3 setup notebook](../00_Intro/bedrock_boto3_setup.ipynb) 노트북을 참조하세요.


In [ ]:
# Make sure you ran `download-dependencies.sh` from the root of the repository first!
%pip install --no-build-isolation --force-reinstall \
    ../dependencies/awscli-*-py3-none-any.whl \
    ../dependencies/boto3-*-py3-none-any.whl \
    ../dependencies/botocore-*-py3-none-any.whl

이 노트북에는 몇 가지 추가 종속성도 필요합니다.

- [FAISS](https://github.com/facebookresearch/faiss), 벡터 임베딩 저장
- [IPyWidgets](https://ipywidgets.readthedocs.io/en/stable/), 노트북에서 대화형 UI 위젯을 사용하기 위해
- [PyPDF](https://pypi.org/project/pypdf/), PDF 파일 처리용

In [ ]:
%pip install --quiet "faiss-cpu>=1.7,<2" "ipywidgets>=7,<8" langchain==0.0.249 "pypdf>=3.8,<4"

In [ ]:
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww


# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
# os.environ["BEDROCK_ENDPOINT_URL"] = "<YOUR_ENDPOINT_URL>"  # E.g. "https://..."


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

## 챗봇(기본 - 컨텍스트 없음)

#### LangChain의 CoversationChain을 사용하여 대화 시작

챗봇은 이전 상호작용을 기억해야 합니다. 대화 기억(Conversational memory)을 사용하면 그렇게 할 수 있습니다. 대화 기억을 구현할 수 있는 방법에는 여러 가지가 있습니다. LangChain의 컨텍스트는 모두 ConversationChain 위에 구축됩니다.

참고: 모델 출력은 비결정적입니다.

In [ ]:
from langchain.chains import ConversationChain
from langchain.llms.bedrock import Bedrock
from langchain.memory import ConversationBufferMemory

titan_llm = Bedrock(model_id="amazon.titan-tg1-large", client=boto3_bedrock)
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=titan_llm, verbose=True, memory=memory
)

print_ww(conversation.predict(input="Hi there!"))

#### 새로운 질문
모델이 초기 메시지로 응답했습니다. 몇 가지 질문을 해보겠습니다.

In [ ]:
print_ww(conversation.predict(input="Give me a few tips on how to start a new garden."))

#### 질문을 토대로 작성

모델이 이전 대화를 이해할 수 있는지 확인하기 위해 정원이라는 단어를 언급하지 않고 질문해 보겠습니다.

In [ ]:
print_ww(conversation.predict(input="Cool. Will that work with tomatoes?"))

#### 대화 마치기

In [ ]:
print_ww(conversation.predict(input="That's all, thank you!"))

## 프롬프트 템플릿(Langchain)을 이용한 챗봇

PromptTemplate은 입력의 구성을 담당합니다. LangChain은 프롬프트를 쉽게 구성하고 작업할 수 있도록 여러 클래스와 함수를 제공합니다. 여기서는 기본 프롬프트 템플릿을 사용하겠습니다. [PromptTemplate](https://python.langchain.com/en/latest/modules/prompts/getting_started.html)

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain import PromptTemplate

chat_history = []

# turn verbose to true to see the full logs and documents
qa= ConversationChain(
    llm=titan_llm, verbose=False, memory=ConversationBufferMemory() #memory_chain
)

print(f"ChatBot:DEFAULT:PROMPT:TEMPLATE: is ={qa.prompt.template}")

In [ ]:
import ipywidgets as ipw
from IPython.display import display, clear_output

class ChatUX:
    """ A chat UX using IPWidgets
    """
    def __init__(self, qa, retrievalChain = False):
        self.qa = qa
        self.name = None
        self.b=None
        self.retrievalChain = retrievalChain
        self.out = ipw.Output()


    def start_chat(self):
        print("Starting chat bot")
        display(self.out)
        self.chat(None)


    def chat(self, _):
        if self.name is None:
            prompt = ""
        else: 
            prompt = self.name.value
        if 'q' == prompt or 'quit' == prompt or 'Q' == prompt:
            print("Thank you , that was a nice chat !!")
            return
        elif len(prompt) > 0:
            with self.out:
                thinking = ipw.Label(value="Thinking...")
                display(thinking)
                try:
                    if self.retrievalChain:
                        result = self.qa.run({'question': prompt })
                    else:
                        result = self.qa.run({'input': prompt }) #, 'history':chat_history})
                except:
                    result = "No answer"
                thinking.value=""
                print_ww(f"AI:{result}")
                self.name.disabled = True
                self.b.disabled = True
                self.name = None

        if self.name is None:
            with self.out:
                self.name = ipw.Text(description="You:", placeholder='q to quit')
                self.b = ipw.Button(description="Send")
                self.b.on_click(self.chat)
                display(ipw.Box(children=(self.name, self.b)))

대화를 시작해봅시다.

In [ ]:
chat = ChatUX(qa)
chat.start_chat()

## 페르소나를 활용한 챗봇

이번 AI 비서는 커리어 코치 역할을 하게 됩니다. 역할극 대화를 위해서는, 채팅을 시작하기 전에 사용자 메시지(역할극을 수행하라는 지시)를 설정해야 합니다. ConversationBufferMemory는 대화 상자를 미리 채우는 데 사용됩니다.

In [ ]:
memory = ConversationBufferMemory()
memory.chat_memory.add_user_message("You will be acting as a career coach. Your goal is to give career advice to users")
memory.chat_memory.add_ai_message("I am career coach and give career advice")
titan_llm = Bedrock(model_id="amazon.titan-tg1-large",client=boto3_bedrock)
conversation = ConversationChain(
     llm=titan_llm, verbose=True, memory=memory
)

print_ww(conversation.predict(input="What are the career options in AI?"))

##### 이 페르소나의 전문영역이 아닌 질문을 해봅니다. 모델은 그 질문에 답하거나 그에 대한 이유를 제시해서는 안됩니다.

In [ ]:
conversation.verbose = False
print_ww(conversation.predict(input="How to fix my car?"))

## 컨텍스트를 활용한 챗봇
이 사용 사례에서는 Chatbot에게 질문이 전달된 컨텍스트에서 답변하도록 요청합니다. csv 파일을 가져와 Titan 임베딩 모델을 사용하여 벡터를 만듭니다. 이 벡터는 FAISS에 저장됩니다. 챗봇이 질문을 받으면 이 벡터를 전달하고 답변을 검색합니다.


#### Titan 임베딩 모델 사용 - 이를 사용하여 문서에 대한 임베딩을 생성할 수 있습니다.

임베딩은 단어, 구문 또는 기타 개별 항목을 연속 벡터 공간의 벡터로 표현하는 방법입니다. 이를 통해 기계 학습 모델은 이러한 표현에 대해 수학적 연산을 수행하고 표현 간의 의미론적 관계를 캡처할 수 있습니다.


이는 RAG [문서 검색 기능](https://labelbox.com/blog/how-Vector-similarity-search-works/)에 사용됩니다.

가능한 다른 임베딩은 여기에 있습니다. [LangChain 임베딩](https://python.langchain.com/en/latest/reference/modules/embeddings.html)

In [ ]:
from langchain.embeddings import BedrockEmbeddings
from langchain.vectorstores import FAISS
from langchain import PromptTemplate

br_embeddings = BedrockEmbeddings(client=boto3_bedrock)

#### 문서 검색을 위한 임베딩 생성

#### 벡터 저장소 인덱서.

인덱서는 임베딩을 저장하고 일치시키는 과정입니다. 이 노트북은 Chroma 및 FAISS를 활용하며, 이는 일시적이며 메모리에 저장됩니다. VectorStore Api는 [여기](https://python.langchain.com/en/harrison-docs-refactor-3-24/reference/modules/Vectorstore.html)에서 확인할 수 있습니다.

임베딩을 반환할 모델을 호출하기 위해 SageMaker 엔드포인트에 대한 참조가 필요한 SageMaker 임베딩의 자체 사용자 지정 구현을 사용하겠습니다. 이는 FAISS 또는 Chroma에서 메모리에 저장하고 사용자가 쿼리를 실행할 때마다 사용됩니다.

#### VectorStore (FAISS)

여기에서 메모리 벡터 스토어 [FAISS](https://arxiv.org/pdf/1702.08734.pdf)에 대해 더 읽어보실 수 있습니다. 그러나 우리의 예에서는 동일합니다.

Chroma

[Chroma](https://www.trychroma.com/)는 초간단 벡터 검색 데이터베이스입니다. 핵심 API는 사용자가 인메모리 문서-벡터 저장소(in-memory document-vector store)를 구축할 수 있도록 하는 4가지 기능으로 구성됩니다. 기본적으로 Chroma는 Hugging Face 변환기 라이브러리를 사용하여 문서를 벡터화합니다.

Weaviate

[Weaviate](https://github.com/weaviate/weaviate)는 매우 고급스러워 보이는 도구입니다. Weaviate는 벡터 검색을 지원하는 GraphQL API를 제공할 뿐만 아니라, 사용자는 Weaviate의 내장 모듈이나 사용자 정의 모듈을 사용하여 콘텐츠를 벡터화할 수 있습니다.


In [ ]:
from langchain.document_loaders import CSVLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.indexes.vectorstore import VectorStoreIndexWrapper

s3_path = f"s3://jumpstart-cache-prod-us-east-2/training-datasets/Amazon_SageMaker_FAQs/Amazon_SageMaker_FAQs.csv"
!aws s3 cp $s3_path ./rag_data/Amazon_SageMaker_FAQs.csv

loader = CSVLoader("./rag_data/Amazon_SageMaker_FAQs.csv") # --- > 219 docs with 400 chars
documents_aws = loader.load() #
print(f"documents:loaded:size={len(documents_aws)}")

docs = CharacterTextSplitter(chunk_size=2000, chunk_overlap=400, separator=",").split_documents(documents_aws)

print(f"Documents:after split and chunking size={len(docs)}")

vectorstore_faiss_aws = FAISS.from_documents(
    documents=docs,
    embedding = br_embeddings, 
    #**k_args
)

print(f"vectorstore_faiss_aws:created={vectorstore_faiss_aws}::")


#### 간단한 테스트를 수행해봅시다.

LangChain에서 제공하는 Wrapper 클래스를 사용하여 벡터 데이터베이스 저장소를 쿼리하고 관련 문서를 반환할 수 있습니다. 뒤에서는 모든 기본값을 사용하여 QA 체인만 실행합니다.

In [ ]:
wrapper_store_faiss = VectorStoreIndexWrapper(vectorstore=vectorstore_faiss_aws)
print_ww(wrapper_store_faiss.query("R in SageMaker", llm=titan_llm))

#### 챗봇 애플리케이션

챗봇의 경우 컨텍스트 관리, 대화 이력, 벡터 저장소 및 기타 여러 가지가 필요합니다. ConversationalRetrievalChain부터 시작하겠습니다.

이는 후속 질문에 사용할 수 있는 채팅 기록 전달을 가능하게 해주는 대화 메모리 및 RetrievalQAChain을 사용합니다. 출처: https://python.langchain.com/en/latest/modules/chains/index_examples/chat_Vector_db.html

뒤에서 무슨 일이 일어나고 있는지 모두 보려면 verbose를 True로 설정하세요.


In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
from langchain.chains import ConversationalRetrievalChain
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT


def create_prompt_template():
    _template = """{chat_history}

Answer only with the new question.
How would you ask the question considering the previous conversation: {question}
Question:"""
    CONVO_QUESTION_PROMPT = PromptTemplate.from_template(_template)
    return CONVO_QUESTION_PROMPT

memory_chain = ConversationBufferMemory(memory_key="chat_history", input_key="question", return_messages=True)
chat_history=[]

#### ConversationRetrievalChain에 사용되는 파라미터
retriever: 우리는 VectorStore가 지원하는 VectoreStoreRetriver를 사용했습니다. 텍스트를 검색하려면 search_type: "similarity" 또는 "mmr"이라는 두 가지 검색 유형을 선택할 수 있습니다. search_type="similarity"는 질문 벡터와 가장 유사한 텍스트 청크 벡터를 선택하는 유사성 검색을 사용합니다.

memory: 히스토리를 저장하는 메모리 체인

condense_question_prompt: 사용자의 질문이 있으면 이전 대화와 해당 질문을 사용하여 독립형 질문을 구성합니다.

chain_type: 채팅 기록이 길고 상황에 맞지 않는 경우 이 매개변수를 사용하고 옵션은 "stuff", "refine", "map_reduce", "map-rerank"입니다.

참고: 질문이 전달된 컨텍스트 범위를 벗어나는 경우 모델은 답변을 모른다고 응답합니다.


In [ ]:
# turn verbose to true to see the full logs and documents
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
from langchain.chains import ConversationalRetrievalChain
qa = ConversationalRetrievalChain.from_llm(
    llm=titan_llm, 
    retriever=vectorstore_faiss_aws.as_retriever(), 
    #retriever=vectorstore_faiss_aws.as_retriever(search_type='similarity', search_kwargs={"k": 8}),
    memory=memory_chain,
    #verbose=True,
    #condense_question_prompt=CONDENSE_QUESTION_PROMPT, # create_prompt_template(), 
    chain_type='stuff', # 'refine',
    #max_tokens_limit=100
)

qa.combine_docs_chain.llm_chain.prompt = PromptTemplate.from_template("""
{context}

Use at maximum 3 sentences to answer the question inside the <q></q> XML tags. 

<q>{question}</q>

Do not use any XML tags in the answer. If the answer is not in the context say "Sorry, I don't know, as the answer was not found in the context."

Answer:""")

대화를 시작해봅시다.

In [ ]:
chat = ChatUX(qa, retrievalChain=True)
chat.start_chat()

### 이 데모에서는 Titan LLM을 사용하여 다음 패턴으로 대화형 인터페이스를 만들었습니다.

1. 챗봇(기본 - 맥락 없음)

2. 프롬프트 템플릿(Langchain)을 이용한 챗봇

3. 페르소나를 갖춘 챗봇

4. 컨텍스트를 활용한 챗봇